In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [2]:
# X = df[['gender', 'age', 'diagnosis']]
# y = df['microbe_type_1']

# model = LinearRegression().fit(X, y)
master_data = '/home/sha/work/microbiome/data/master_tax_1.csv'
master = pd.read_csv(master_data)

In [3]:
master

,Unnamed: 0,Subject_ID,Visit_number,Visit_ID,Age,Sex,Diagnosis,Race,g__Abiotrophia,g__Acidaminococcaceae_unclassified,...,g__Tetragenococcus,g__Treponema,g__Tunalikevirus,g__Turicibacter,g__Ureaplasma,g__Varibaculum,g__Veillonella,g__Weissella,g__Wohlfahrtiimonas,g__Yersinia
0,0,2008,23,MSM5LLDE,30,Female,CD,White,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.05505,0.0,0.0,0.0
1,1,2008,6,MSM5LLDM,30,Female,CD,White,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
2,2,2008,28,MSM6J2LB,30,Female,CD,White,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,3,2008,11,MSM5LLDU,30,Female,CD,White,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,4,2008,22,MSM5LLDC,30,Female,CD,White,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,1308,3022,19,CSM7KOOH,69,Male,nonIBD,White,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.07603,0.0,0.0,0.0
1309,1309,3022,4,CSM67UH7,69,Male,nonIBD,White,0.0,0.00249,...,0.0,0.0,0.0,0.0,0.0,0.0,0.01277,0.0,0.0,0.0
1310,1310,3022,22,CSM7KOON,69,Male,nonIBD,White,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1311,1311,3022,25,CSM7KOOR,69,Male,nonIBD,White,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


In [4]:
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

In [5]:
m1 = master
le = LabelEncoder()
m1['Visit_ID'] = le.fit_transform(m1['Visit_ID'])
cols = ['Visit_ID', 'Sex', 'Diagnosis', 'Race']
for col in cols:
    m1[col] = le.fit_transform(m1[col])
m1
m1.corr()

,Unnamed: 0,Subject_ID,Visit_number,Visit_ID,Age,Sex,Diagnosis,Race,g__Abiotrophia,g__Acidaminococcaceae_unclassified,...,g__Tetragenococcus,g__Treponema,g__Tunalikevirus,g__Turicibacter,g__Ureaplasma,g__Varibaculum,g__Veillonella,g__Weissella,g__Wohlfahrtiimonas,g__Yersinia
Unnamed: 0,1.000000,-0.125372,-0.076674,-0.092499,0.222402,0.092476,0.030906,0.056574,-0.002736,0.089592,...,0.006847,-0.024911,-0.030889,0.049838,-0.036419,-0.062499,-0.130750,-0.047745,-0.055337,-0.050896
Subject_ID,-0.125372,1.000000,-0.053222,0.250426,-0.585789,0.158563,-0.171382,-0.139362,0.011069,-0.001285,...,0.051275,0.010058,-0.004244,0.007618,-0.031703,0.063125,0.004765,-0.028055,-0.038316,-0.010748
Visit_number,-0.076674,-0.053222,1.000000,0.003903,0.038122,-0.037171,-0.030819,-0.039066,0.010414,-0.019385,...,0.035243,0.024136,0.030593,0.036964,0.042648,0.008794,0.010171,0.005104,0.040072,0.012125
Visit_ID,-0.092499,0.250426,0.003903,1.000000,-0.274270,0.251663,0.236951,0.034763,-0.017823,-0.098470,...,0.037730,0.004225,-0.037438,-0.007709,0.018355,0.068350,0.020234,-0.011552,0.014170,0.021970
Age,0.222402,-0.585789,0.038122,-0.274270,1.000000,0.005539,0.168116,0.119429,-0.024550,0.175693,...,-0.026432,-0.017241,0.024713,-0.006921,-0.008051,-0.037920,0.012591,0.039772,0.005014,-0.001925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
g__Varibaculum,-0.062499,0.063125,0.008794,0.068350,-0.037920,0.029635,-0.040028,0.011948,-0.001583,-0.006553,...,-0.001305,-0.001305,-0.001809,-0.004437,0.090366,1.000000,-0.011876,-0.001690,-0.001557,-0.001697
g__Veillonella,-0.130750,0.004765,0.010171,0.020234,0.012591,0.031411,-0.094542,-0.061143,-0.007131,-0.028939,...,-0.006935,-0.002822,0.196184,0.017761,-0.006935,-0.011876,1.000000,0.082528,-0.007106,-0.003957
g__Weissella,-0.047745,-0.028055,0.005104,-0.011552,0.039772,0.053914,0.007605,0.007772,0.067944,-0.010712,...,0.022919,-0.002134,0.300403,-0.007253,-0.002134,-0.001690,0.082528,1.000000,-0.002545,-0.002774
g__Wohlfahrtiimonas,-0.055337,-0.038316,0.040072,0.014170,0.005014,0.032067,-0.032629,0.008322,-0.001103,-0.004564,...,-0.000909,-0.000909,-0.001260,-0.003090,-0.000909,-0.001557,-0.007106,-0.002545,1.000000,0.017510


In [6]:
ratio = 0.2

col = 'Subject_ID'
sub = m1.corr()[col]
for i in range(len(sub)):
    if abs(sub[i]) > ratio:
        print(f"{col}: {i} {m1.columns[7+i]}")

col = 'Visit_ID'
sub = m1.corr()['Visit_ID']
for i in range(len(sub)):
    if abs(sub[i]) > ratio:
        print(f"{col}: {i} {m1.columns[7+i]}")

col = 'Age'
sub = m1.corr()['Age']
for i in range(len(sub)):
    if abs(sub[i]) > ratio:
        print(f"{col}: {i} {m1.columns[7+i]}")

col = 'Sex'
sub = m1.corr()['Sex']
for i in range(len(sub)):
    if abs(sub[i]) > 0.5:
        print(f"{col}: {i} {m1.columns[7+i]}")

col = 'Race'
sub = m1.corr()['Race']
for i in range(len(sub)):
    if abs(sub[i]) > 0.5:
        print(f"{col}: {i} {m1.columns[7+i]}")

col = 'Diagnosis'
sub = m1.corr()['Diagnosis']
for i in range(len(sub)):
    if abs(sub[i]) > 0.5:
        print(f"{col}: {i} {m1.columns[7+i]}")

Subject_ID: 1 g__Abiotrophia
Subject_ID: 3 g__Acidaminococcus
Subject_ID: 4 g__Acinetobacter
Visit_ID: 1 g__Abiotrophia
Visit_ID: 3 g__Acidaminococcus
Visit_ID: 4 g__Acinetobacter
Visit_ID: 5 g__Actinobacillus
Visit_ID: 6 g__Actinobaculum
Visit_ID: 31 g__Bordetella
Age: 0 Race
Age: 1 g__Abiotrophia
Age: 3 g__Acidaminococcus
Age: 4 g__Acinetobacter
Age: 182 g__Tunalikevirus
Age: 185 g__Varibaculum
Sex: 5 g__Actinobacillus
Race: 7 g__Actinomyces
Diagnosis: 6 g__Actinobaculum


In [7]:
X = m1[['Subject_ID', 'Visit_ID', 'Age', 'Sex', 'Diagnosis', 'Race']]
abundances = m1.iloc[:, 8:]

In [8]:
for i in range(len(abundances.columns)):
    y = abundances.iloc[:, i]
    model = LinearRegression().fit(X, y)
    scores = cross_val_score(model, X, y, cv=5)
    print("i: ", i, scores.mean())

i:  0 -5.0429981957067005
i:  1 -0.5464862206022565
i:  2 -0.9289641014381036
i:  3 -0.0007633587786259443
i:  4 -3.51862672729641
i:  5 -0.0007633587786260332
i:  6 -92.04159371765186
i:  7 -0.7837142663947666
i:  8 -0.001251378954914939
i:  9 -0.754719661270911
i:  10 -0.029736593483903583
i:  11 -0.045642977204244685
i:  12 -5.541344969201644
i:  13 -0.024124198483539906
i:  14 -1320.1785647846807
i:  15 -25.48125728074744
i:  16 -0.0015229281294056652
i:  17 -0.15098119085093367
i:  18 -0.2538327125674402
i:  19 -0.0015277795701522212
i:  20 -0.0007633587786259
i:  21 -49.50561328802453
i:  22 -0.2608969509725351
i:  23 -0.17487201891398202
i:  24 -0.0015833118024760396
i:  25 -0.10749894927979049
i:  26 -0.0007633587786259443
i:  27 -0.07991257511523524
i:  28 -0.2501206580618413
i:  29 -0.02789422069604557
i:  30 -0.0007662835249041766
i:  31 -0.0007633587786259443
i:  32 -0.25287565343438095
i:  33 -0.6037144389125377
i:  34 -0.011977722778072986
i:  35 -0.2552889834842792
i:  3

In [9]:
abundances

,g__Abiotrophia,g__Acidaminococcaceae_unclassified,g__Acidaminococcus,g__Acinetobacter,g__Actinobacillus,g__Actinobaculum,g__Actinomyces,g__Adlercreutzia,g__Aerococcus,g__Aggregatibacter,...,g__Tetragenococcus,g__Treponema,g__Tunalikevirus,g__Turicibacter,g__Ureaplasma,g__Varibaculum,g__Veillonella,g__Weissella,g__Wohlfahrtiimonas,g__Yersinia
0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.05505,0.0,0.0,0.0
1,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
2,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.07603,0.0,0.0,0.0
1309,0.0,0.00249,0.0,0.0,0.0,0.0,0.0,0.01412,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.01277,0.0,0.0,0.0
1310,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
1311,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


In [10]:
cols = ['Subject_ID', 'Visit_ID', 'Sex', 'Diagnosis', 'Race']

In [11]:
X = abundances
for i in range(len(cols)):
    y = master.loc[:, cols[i]]
    # model = LinearRegression().fit(X, y)
    # scores = cross_val_score(model, X, y, cv=5)
    # print("i:", i, scores.mean())
    selector = SelectKBest(chi2, k=6)
    X_new = selector.fit_transform(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=2222)
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"{cols[i]}'s Accuracy: {accuracy}")

/home/sha/work/_p310/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Subject_ID's Accuracy: 0.1520912547528517


/home/sha/work/_p310/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Visit_ID's Accuracy: 0.0
Sex's Accuracy: 0.5475285171102662
Diagnosis's Accuracy: 0.5399239543726235
Race's Accuracy: 0.908745247148289
